### Mongodb Eduhub

In [4]:
## Importing libraries
from pymongo import MongoClient
from faker import Faker
from datetime import datetime, timezone, timedelta
import pandas as pd

In [20]:
import uuid
import random


In [6]:
# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]

In [ ]:
# users Collection — with Validation Rules
db.create_collection(
    "users",
    validator={
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["userId", "email", "firstName", "lastName", "role"],
            "properties": {
                "userId": {"bsonType": "string"},
                "email": {
                    "bsonType": "string",
                    "pattern": "^.+@.+$"
                },
                "firstName": {"bsonType": "string"},
                "lastName": {"bsonType": "string"},
                "role": {
                    "enum": ["student", "instructor"]
                },
                "dateJoined": {"bsonType": "date"},
                "profile": {
                    "bsonType": "object",
                    "properties": {
                        "bio": {"bsonType": "string"},
                        "avatar": {"bsonType": "string"},
                        "skills": {
                            "bsonType": "array",
                            "items": {"bsonType": "string"}
                        }
                    }
                },
                "isActive": {"bsonType": "bool"}
            }
        }
    }
)


In [ ]:
# courses Collection — with Validation Rules
db.create_collection(
    "courses",
    validator={
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["courseId", "title", "instructorId", "level"],
            "properties": {
                "courseId": {"bsonType": "string"},
                "title": {"bsonType": "string"},
                "description": {"bsonType": "string"},
                "instructorId": {"bsonType": "string"},
                "category": {"bsonType": "string"},
                "level": {
                    "enum": ["beginner", "intermediate", "advanced"]
                },
                "duration": {"bsonType": "double"},
                "price": {"bsonType": "double"},
                "tags": {
                    "bsonType": "array",
                    "items": {"bsonType": "string"}
                },
                "createdAt": {"bsonType": "date"},
                "updatedAt": {"bsonType": "date"},
                "isPublished": {"bsonType": "bool"},
                "rating": {"bsonType": "double"},
            }
        }
    }
)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'courses')

In [ ]:
enrollments_schema = {
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["enrollmentId", "studentId", "courseId", "enrolledAt", "status"],
            "properties": {
                "enrollmentId": {"bsonType": "string"},
                "studentId": {"bsonType": "string"},  # Reference to users.userId
                "courseId": {"bsonType": "string"},   # Reference to courses.courseId
                "enrolledAt": {"bsonType": "date"},
                "status": {
                    "enum": ["active", "completed", "dropped"]
                },
                "progress": {
                    "bsonType": "double",
                    "minimum": 0,
                    "maximum": 100
                }
            }
        }
    }
}

lessons_schema = {
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["lessonId", "courseId", "title", "content", "order"],
            "properties": {
                "lessonId": {"bsonType": "string"},
                "courseId": {"bsonType": "string"},  # Reference to courses.courseId
                "title": {"bsonType": "string"},
                "content": {"bsonType": "string"},
                "resources": {
                    "bsonType": "array",
                    "items": {"bsonType": "string"}
                },
                "order": {"bsonType": "int"},  # Lesson sequence
                "createdAt": {"bsonType": "date"}
            }
        }
    }
}

assignments_schema = {
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["assignmentId", "courseId", "title", "description", "dueDate"],
            "properties": {
                "assignmentId": {"bsonType": "string"},
                "courseId": {"bsonType": "string"},  # Reference to courses.courseId
                "lessonId": {"bsonType": "string"},  # Optional, for lesson-specific assignments
                "title": {"bsonType": "string"},
                "description": {"bsonType": "string"},
                "dueDate": {"bsonType": "date"},
                "points": {"bsonType": "int"},
                "createdAt": {"bsonType": "date"}
            }
        }
    }
}

submissions_schema = {
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["submissionId", "assignmentId", "studentId", "submittedAt"],
            "properties": {
                "submissionId": {"bsonType": "string"},
                "assignmentId": {"bsonType": "string"},  # Reference to assignments.assignmentId
                "studentId": {"bsonType": "string"},     # Reference to users.userId
                "content": {"bsonType": "string"},
                "grade": {"bsonType": "double"},
                "feedback": {"bsonType": "string"},
                "submittedAt": {"bsonType": "date"}
            }
        }
    }
}


In [ ]:
db.create_collection("enrollments", validator=enrollments_schema["validator"])
db.create_collection("lessons", validator=lessons_schema["validator"])
db.create_collection("assignments", validator=assignments_schema["validator"])
db.create_collection("submissions", validator=submissions_schema["validator"])


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eduhub_db'), 'submissions')

### Section Title: Task 1.2 – Document Schema Design

In [ ]:
# Users Collection Schema
user_schema = {
    "_id": "ObjectId (auto-generated)",
    "userId": "string (unique)",
    "email": "string (unique, required)",
    "firstName": "string (required)",
    "lastName": "string (required)",
    "role": "string (enum: ['student', 'instructor'])",
    "dateJoined": "datetime",
    "profile": {
        "bio": "string",
        "avatar": "string",
        "skills": ["string"]
    },
    "isActive": "boolean"
}

# Courses Collection Schema
course_schema = {
    "_id": "ObjectId (auto-generated)",
    "courseId": "string (unique)",
    "title": "string (required)",
    "description": "string",
    "instructorId": "string (reference to users)",
    "category": "string",
    "level": "string (enum: ['beginner', 'intermediate', 'advanced'])",
    "duration": "number (in hours)",
    "price": "number",
    "tags": ["string"],
    "createdAt": "datetime",
    "updatedAt": "datetime",
    "isPublished": "boolean",
    "rating": "number"
}

# Enrollments Collection Schema
enrollment_schema = {
    "_id": "ObjectId (auto-generated)",
    "enrollmentId": "string (unique)",
    "studentId": "string (reference to users)",
    "courseId": "string (reference to courses)",
    "enrolledAt": "datetime",
    "status": "string (enum: ['active', 'completed', 'dropped'])",
    "progress": "number"
}

# Lessons Collection Schema
lesson_schema = {
    "_id": "ObjectId (auto-generated)",
    "lessonId": "string (unique)",
    "courseId": "string (reference to courses)",
    "title": "string (required)",
    "content": "string",
    "resources": ["string"],
    "order": "number",
    "createdAt": "datetime"
}

# Assignments Collection Schema
assignment_schema = {
    "_id": "ObjectId (auto-generated)",
    "assignmentId": "string (unique)",
    "courseId": "string (reference to courses)",
    "lessonId": "string (reference to lessons)",
    "title": "string (required)",
    "description": "string",
    "dueDate": "datetime",
    "points": "number",
    "createdAt": "datetime"
}

# Submissions Collection Schema
submission_schema = {
    "_id": "ObjectId (auto-generated)",
    "submissionId": "string (unique)",
    "assignmentId": "string (reference to assignments)",
    "studentId": "string (reference to users)",
    "content": "string",
    "grade": "number",
    "feedback": "string",
    "submittedAt": "datetime"
}


Task 2.1: Insert Sample Data

In [21]:
faker = Faker()



In [ ]:
#Inserting sample data in users collection
roles = ["student"] * 14 + ["instructor"] * 6
random.shuffle(roles)

users = []
for i in range(20):
    user = {
        "userId": str(uuid.uuid4()),
        "email": faker.email(),
        "firstName": faker.first_name(),
        "lastName": faker.last_name(),
        "role": roles[i],
        "dateJoined": faker.date_time_this_year(),
        "profile": {
            "bio": faker.text(150),
            "avatar": faker.image_url(),
            "skills": [faker.word() for _ in range(random.randint(1, 4))]
        },
        "isActive": random.choice([True, False])
    }
    users.append(user)

db.users.insert_many(users)


InsertManyResult([ObjectId('684af3ecf4735c5cb4158443'), ObjectId('684af3ecf4735c5cb4158444'), ObjectId('684af3ecf4735c5cb4158445'), ObjectId('684af3ecf4735c5cb4158446'), ObjectId('684af3ecf4735c5cb4158447'), ObjectId('684af3ecf4735c5cb4158448'), ObjectId('684af3ecf4735c5cb4158449'), ObjectId('684af3ecf4735c5cb415844a'), ObjectId('684af3ecf4735c5cb415844b'), ObjectId('684af3ecf4735c5cb415844c'), ObjectId('684af3ecf4735c5cb415844d'), ObjectId('684af3ecf4735c5cb415844e'), ObjectId('684af3ecf4735c5cb415844f'), ObjectId('684af3ecf4735c5cb4158450'), ObjectId('684af3ecf4735c5cb4158451'), ObjectId('684af3ecf4735c5cb4158452'), ObjectId('684af3ecf4735c5cb4158453'), ObjectId('684af3ecf4735c5cb4158454'), ObjectId('684af3ecf4735c5cb4158455'), ObjectId('684af3ecf4735c5cb4158456')], acknowledged=True)

In [23]:
# Inserting sample data in courses collection

instructors = [u for u in users if u["role"] == "instructor"]
categories = ["Data Science", "Web Dev", "Business", "AI", "Cybersecurity"]
levels = ["beginner", "intermediate", "advanced"]

courses = []
for _ in range(8):
    instructor = random.choice(instructors)
    course = {
        "courseId": str(uuid.uuid4()),
        "title": faker.catch_phrase(),
        "description": faker.text(150),
        "instructorId": instructor["userId"],
        "category": random.choice(categories),
        "level": random.choice(levels),
        "duration": round(random.uniform(1.5, 20.0), 1),
        "price": round(random.uniform(10, 100), 2),
        "tags": faker.words(nb=3),
        "createdAt": faker.date_time_this_year(),
        "updatedAt": faker.date_time_this_year(),
        "isPublished": random.choice([True, False])
    }
    courses.append(course)

db.courses.insert_many(courses)


InsertManyResult([ObjectId('684af49bf4735c5cb4158457'), ObjectId('684af49bf4735c5cb4158458'), ObjectId('684af49bf4735c5cb4158459'), ObjectId('684af49bf4735c5cb415845a'), ObjectId('684af49bf4735c5cb415845b'), ObjectId('684af49bf4735c5cb415845c'), ObjectId('684af49bf4735c5cb415845d'), ObjectId('684af49bf4735c5cb415845e')], acknowledged=True)

In [24]:
## Inserting 15 Enrollments (studentId + courseId)
students = [u for u in users if u["role"] == "student"]
statuses = ["active", "completed", "dropped"]

enrollments = []
for _ in range(15):
    student = random.choice(students)
    course = random.choice(courses)
    enrollment = {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": student["userId"],
        "courseId": course["courseId"],
        "enrolledAt": faker.date_time_this_year(),
        "status": random.choice(statuses),
        "progress": round(random.uniform(0, 100), 2)
    }
    enrollments.append(enrollment)

db.enrollments.insert_many(enrollments)


InsertManyResult([ObjectId('684af514f4735c5cb415845f'), ObjectId('684af514f4735c5cb4158460'), ObjectId('684af514f4735c5cb4158461'), ObjectId('684af514f4735c5cb4158462'), ObjectId('684af514f4735c5cb4158463'), ObjectId('684af514f4735c5cb4158464'), ObjectId('684af514f4735c5cb4158465'), ObjectId('684af514f4735c5cb4158466'), ObjectId('684af514f4735c5cb4158467'), ObjectId('684af514f4735c5cb4158468'), ObjectId('684af514f4735c5cb4158469'), ObjectId('684af514f4735c5cb415846a'), ObjectId('684af514f4735c5cb415846b'), ObjectId('684af514f4735c5cb415846c'), ObjectId('684af514f4735c5cb415846d')], acknowledged=True)

In [25]:
# Inserting sample data in lessons collection
lessons = []
for _ in range(25):
    course = random.choice(courses)
    lesson = {
        "lessonId": str(uuid.uuid4()),
        "courseId": course["courseId"],
        "title": faker.sentence(),
        "content": faker.text(300),
        "resources": [faker.url() for _ in range(random.randint(0, 3))],
        "order": random.randint(1, 10),
        "createdAt": faker.date_time_this_year()
    }
    lessons.append(lesson)

db.lessons.insert_many(lessons)


InsertManyResult([ObjectId('684af628f4735c5cb415846e'), ObjectId('684af628f4735c5cb415846f'), ObjectId('684af628f4735c5cb4158470'), ObjectId('684af628f4735c5cb4158471'), ObjectId('684af628f4735c5cb4158472'), ObjectId('684af628f4735c5cb4158473'), ObjectId('684af628f4735c5cb4158474'), ObjectId('684af628f4735c5cb4158475'), ObjectId('684af628f4735c5cb4158476'), ObjectId('684af628f4735c5cb4158477'), ObjectId('684af628f4735c5cb4158478'), ObjectId('684af628f4735c5cb4158479'), ObjectId('684af628f4735c5cb415847a'), ObjectId('684af628f4735c5cb415847b'), ObjectId('684af628f4735c5cb415847c'), ObjectId('684af628f4735c5cb415847d'), ObjectId('684af628f4735c5cb415847e'), ObjectId('684af628f4735c5cb415847f'), ObjectId('684af628f4735c5cb4158480'), ObjectId('684af628f4735c5cb4158481'), ObjectId('684af628f4735c5cb4158482'), ObjectId('684af628f4735c5cb4158483'), ObjectId('684af628f4735c5cb4158484'), ObjectId('684af628f4735c5cb4158485'), ObjectId('684af628f4735c5cb4158486')], acknowledged=True)

In [30]:
# Inserting sample data in assignments collection
assignments = []

for _ in range(10):  # Generate 10 sample assignments
    course = random.choice(courses)
    
    assignment = {
        "assignmentId": str(uuid.uuid4()),
        "courseId": course["courseId"],
        "title": faker.sentence(nb_words=6),
        "description": faker.text(max_nb_chars=100),
        "dueDate": faker.future_datetime(end_date="+30d"),
        "points": faker.random_int(min=5, max=20),
        "createdAt": faker.date_time_this_year()
    }

    # Add optional lessonId only if lessons exist and randomly chosen
    if lessons and random.choice([True, False]):
        lesson = random.choice(lessons)
        if "lessonId" in lesson:
            assignment["lessonId"] = lesson["lessonId"]

    assignments.append(assignment)
db.assignments.insert_many(assignments)


InsertManyResult([ObjectId('684af872f4735c5cb415849b'), ObjectId('684af872f4735c5cb415849c'), ObjectId('684af872f4735c5cb415849d'), ObjectId('684af872f4735c5cb415849e'), ObjectId('684af872f4735c5cb415849f'), ObjectId('684af872f4735c5cb41584a0'), ObjectId('684af872f4735c5cb41584a1'), ObjectId('684af872f4735c5cb41584a2'), ObjectId('684af872f4735c5cb41584a3'), ObjectId('684af872f4735c5cb41584a4')], acknowledged=True)

In [31]:
# Inserting sample data in submissions collection
submissions = []
for _ in range(12):
    student = random.choice(students)
    assignment = random.choice(assignments)
    submission = {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": assignment["assignmentId"],
        "studentId": student["userId"],
        "content": faker.paragraph(nb_sentences=5),
        "grade": round(random.uniform(0, 100), 2),
        "feedback": faker.sentence(),
        "submittedAt": faker.date_time_this_year()
    }
    submissions.append(submission)

db.submissions.insert_many(submissions)


InsertManyResult([ObjectId('684af8bdf4735c5cb41584a5'), ObjectId('684af8bdf4735c5cb41584a6'), ObjectId('684af8bdf4735c5cb41584a7'), ObjectId('684af8bdf4735c5cb41584a8'), ObjectId('684af8bdf4735c5cb41584a9'), ObjectId('684af8bdf4735c5cb41584aa'), ObjectId('684af8bdf4735c5cb41584ab'), ObjectId('684af8bdf4735c5cb41584ac'), ObjectId('684af8bdf4735c5cb41584ad'), ObjectId('684af8bdf4735c5cb41584ae'), ObjectId('684af8bdf4735c5cb41584af'), ObjectId('684af8bdf4735c5cb41584b0')], acknowledged=True)

### Section 3: Basic CRUD Operations

In [33]:
### Section 3.1 Create Operations
# 1. Add a new student user
new_student = {
    "userId": str(uuid.uuid4()),
    "email": "newstudent@example.com",
    "firstName": "Ada",
    "lastName": "Okonkwo",
    "role": "student",
    "dateJoined": datetime.now(timezone.utc),
    "isActive": True,
    "profile": {
        "bio": "Excited to learn!",
        "avatar": "https://example.com/avatar/ada.jpg",  # URL or path to avatar image
        "skills": ["Python", "Data Analysis"]
    }
}
db.users.insert_one(new_student)

# 2. Create a new course
new_course = {
    "courseId": str(uuid.uuid4()),
    "title": "Data Analysis with Python",
    "description": "Learn how to analyze data using Python.",
    "instructorId": db.users.find_one({"role": "instructor"})["userId"],
    "category": "Data Science",
    "level": "beginner",
    "duration": 15.0,
    "price": 0.0,
    "tags": ["data", "python"],
    "createdAt": datetime.now(timezone.utc),
    "updatedAt": datetime.now(timezone.utc),
    "isPublished": False
}
db.courses.insert_one(new_course)

# 3. Enroll a student in a course
new_enrollment = {
    "enrollmentId": str(uuid.uuid4()),
    "studentId": new_student["userId"],
    "courseId": new_course["courseId"],
    "enrolledAt": datetime.now(timezone.utc),
    "status": "active",
    "progress": 0.0
}
db.enrollments.insert_one(new_enrollment)

# 4. Add a new lesson to an existing course
new_lesson = {
    "lessonId": str(uuid.uuid4()),
    "courseId": new_course["courseId"],
    "title": "Introduction to Data Analysis",
    "content": "This lesson introduces basic concepts in data analysis.",
    "resources": ["https://docs.python.org"],
    "order": 1,
    "createdAt": datetime.now(timezone.utc)
}
db.lessons.insert_one(new_lesson)

InsertOneResult(ObjectId('684afcdef4735c5cb41584b4'), acknowledged=True)

In [34]:
### Section 3.2 Read Operations
# 1. Find all active students
active_students = list(db.users.find({"role": "student", "isActive": True}))

# 2. Retrieve course details with instructor info
course_with_instructor = db.courses.aggregate([
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    },
    {"$unwind": "$instructor"}
])
print(list(course_with_instructor))

# 3. Get all courses in a specific category
data_science_courses = list(db.courses.find({"category": "Data Science"}))

# 4. Find students enrolled in a particular course
course_id = new_course["courseId"]
student_ids = db.enrollments.find({"courseId": course_id}, {"studentId": 1})
student_ids = [enr["studentId"] for enr in student_ids]
students = list(db.users.find({"userId": {"$in": student_ids}}))

# 5. Search courses by title (case-insensitive, partial match)
search_term = "data"
matched_courses = list(db.courses.find({"title": {"$regex": search_term, "$options": "i"}}))


[{'_id': ObjectId('684af49bf4735c5cb4158457'), 'courseId': 'd0023b00-35a7-4fd7-97c1-d397bda85125', 'title': 'Cloned exuding paradigm', 'description': 'Memory former history because behind.\nCertain every benefit you. Computer truth modern serve business drug. Start far light Congress provide may.', 'instructorId': '70f51a9d-d94b-469e-a5cf-bf95f15917c4', 'category': 'Cybersecurity', 'level': 'advanced', 'duration': 3.2, 'price': 39.03, 'tags': ['choose', 'upon', 'we'], 'createdAt': datetime.datetime(2025, 2, 8, 9, 4, 46, 264000), 'updatedAt': datetime.datetime(2025, 5, 31, 18, 4, 26, 940000), 'isPublished': True, 'instructor': {'_id': ObjectId('684af3ecf4735c5cb4158448'), 'userId': '70f51a9d-d94b-469e-a5cf-bf95f15917c4', 'email': 'michelledyer@example.org', 'firstName': 'Susan', 'lastName': 'Dunn', 'role': 'instructor', 'dateJoined': datetime.datetime(2025, 5, 20, 1, 38, 6, 453000), 'profile': {'bio': 'Something act decision require. Door on agreement blue quickly.\nWindow minute enjoy 

In [ ]:
### Section 3.3: Update Operatiosn
# 1. Update a user's profile information
db.users.update_one(
    {"userId": new_student["userId"]},
    {"$set": {"profile.bio": "Updated bio for student", "profile.skills": ["Python", "MongoDB"]}}
)

# 2. Mark a course as published
db.courses.update_one(
    {"courseId": new_course["courseId"]},
    {"$set": {"isPublished": True, "updatedAt": datetime.utcnow()}}
)

# 3. Update assignment grades
submission = db.submissions.find_one()
db.submissions.update_one(
    {"submissionId": submission["submissionId"]},
    {"$set": {"grade": 85.0, "feedback": "Great work!"}}
)

# 4. Add tags to an existing course
db.courses.update_one(
    {"courseId": new_course["courseId"]},
    {"$addToSet": {"tags": {"$each": ["analysis", "beginner"]}}}
)


UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [37]:
## Section 3.3: Delete Operations
# 1. Remove a user (soft delete)
db.users.update_one(
    {"userId": new_student["userId"]},
    {"$set": {"isActive": False}}
)

# 2. Delete an enrollment
db.enrollments.delete_one({"studentId": new_student["userId"], "courseId": new_course["courseId"]})

# 3. Remove a lesson from a course
db.lessons.delete_one({"courseId": new_course["courseId"], "order": 1})


DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

### Section 4: Advanced Queries and Aggregation

In [ ]:
## Section 4.1 Complex Queries
## 1. Find courses with price between $50 and $200
courses_in_price_range = list(db.courses.find({
    "price": {"$gte": 50, "$lte": 200}
}))

## 2. Get users who joined in the last 6 months
from datetime import timedelta

six_months_ago = datetime.now(timezone.utc) - timedelta(days=6*30)  # approx 6 months
recent_users = list(db.users.find({
    "dateJoined": {"$gte": six_months_ago}
}))

## 3. Find courses that have specific tags using $in operator
tags_to_search = ["python", "data science"]

courses_with_tags = list(db.courses.find({
    "tags": {"$in": tags_to_search}
}))

## 4. Retrieve assignments with due dates in the next week
today = datetime.now(timezone.utc)
next_week = today + timedelta(days=7)

upcoming_assignments = list(db.assignments.find({
    "dueDate": {"$gte": today, "$lte": next_week}
}))


In [55]:
courses_in_price_range

[{'_id': ObjectId('684af49bf4735c5cb4158458'),
  'courseId': '74605146-5400-446c-8638-3e268cf0ecfc',
  'title': 'Switchable 6thgeneration open architecture',
  'description': 'Anyone likely necessary.\nIncrease section weight foot. Must hair design ahead rich.',
  'instructorId': 'dd10ef06-389c-4e00-a33f-d941bc6a5cc1',
  'category': 'Data Science',
  'level': 'advanced',
  'duration': 14.2,
  'price': 90.41,
  'tags': ['federal', 'soon', 'knowledge'],
  'createdAt': datetime.datetime(2025, 2, 4, 19, 31, 59, 916000),
  'updatedAt': datetime.datetime(2025, 6, 8, 10, 39, 26, 93000),
  'isPublished': False},
 {'_id': ObjectId('684af49bf4735c5cb415845a'),
  'courseId': '31947696-320f-4ffb-a913-08a320e1e327',
  'title': 'Inverse logistical benchmark',
  'description': 'Explain area city happen how. Mean quickly moment machine reveal. Realize tend section crime.',
  'instructorId': '7cd1ab9a-a9a8-475b-9264-4b867caf783b',
  'category': 'Cybersecurity',
  'level': 'beginner',
  'duration': 17.0

In [56]:
six_months_ago

datetime.datetime(2024, 12, 14, 16, 44, 10, 219925, tzinfo=datetime.timezone.utc)

In [57]:
courses_with_tags

[{'_id': ObjectId('684afcdef4735c5cb41584b2'),
  'courseId': '04cb0722-7fa2-43d6-bc65-0c1cd067ed3b',
  'title': 'Data Analysis with Python',
  'description': 'Learn how to analyze data using Python.',
  'instructorId': '70f51a9d-d94b-469e-a5cf-bf95f15917c4',
  'category': 'Data Science',
  'level': 'beginner',
  'duration': 15.0,
  'price': 0.0,
  'tags': ['data', 'python', 'analysis', 'beginner'],
  'createdAt': datetime.datetime(2025, 6, 12, 16, 14, 22, 212000),
  'updatedAt': datetime.datetime(2025, 6, 12, 16, 17, 58, 673000),
  'isPublished': True}]

In [58]:
upcoming_assignments

[{'_id': ObjectId('684af872f4735c5cb415849f'),
  'assignmentId': '73d170cc-adc9-44c9-bd40-40b77a18da9d',
  'courseId': '31947696-320f-4ffb-a913-08a320e1e327',
  'title': 'Society student sport decade performance six include.',
  'description': 'National marriage worker strategy environmental. Feel unit Congress enjoy left.',
  'dueDate': datetime.datetime(2025, 6, 18, 8, 25, 48, 688000),
  'points': 12,
  'createdAt': datetime.datetime(2025, 6, 11, 16, 43, 41, 526000)},
 {'_id': ObjectId('684af872f4735c5cb41584a1'),
  'assignmentId': '2aed03a4-2c55-45d2-a8a8-dc065fbd370a',
  'courseId': 'ad439ff4-7fa2-43ef-aeff-fea996ca1d4f',
  'title': 'Week goal rule nature beat.',
  'description': 'Medical western over make whole financial compare. Second remain town pull sort purpose.',
  'dueDate': datetime.datetime(2025, 6, 12, 21, 37, 44, 12000),
  'points': 20,
  'createdAt': datetime.datetime(2025, 4, 20, 6, 15, 25, 963000),
  'lessonId': '9f030611-f077-4141-96b7-05279b8f72af'},
 {'_id': Objec

In [ ]:
## Section 4.2: Aggregation Pipelines
## 1a. Course Enrollment Statistics
enrollments_per_course = list(db.enrollments.aggregate([
    {
        "$group": {
            "_id": "$courseId",
            "totalEnrollments": {"$sum": 1}
        }
    }
]))
enrollments_per_course


[{'_id': '74605146-5400-446c-8638-3e268cf0ecfc', 'totalEnrollments': 4},
 {'_id': '31947696-320f-4ffb-a913-08a320e1e327', 'totalEnrollments': 3},
 {'_id': '0b1598d1-e7cf-49de-9f4c-cba8d279698b', 'totalEnrollments': 4},
 {'_id': 'd0023b00-35a7-4fd7-97c1-d397bda85125', 'totalEnrollments': 3},
 {'_id': 'b97fe9ff-2312-4045-876c-b71e4f277d68', 'totalEnrollments': 1}]

In [ ]:
## b. Calculate average course rating
## Updating courses collections to include ratings first


# Fetch all courses
courses = list(db.courses.find({}))

for course in courses:
    rating = round(random.uniform(1, 5), 1)  # e.g., 3.7
    db.courses.update_one(
        {"_id": course["_id"]},
        {"$set": {"rating": rating}}
    )
print("Assigned random ratings to all courses.")

Assigned random ratings to all courses.


In [63]:
avg_course_rating = list(db.courses.aggregate([
    {
        "$unwind": "$rating"
    },
    {
        "$group": {
            "_id": "$courseId",
            "avgRating": {"$avg": "$rating"}
        }
    }
]))
avg_course_rating

[{'_id': '5f1d3c4d-1ca2-444b-bf32-49e1332dfbe6', 'avgRating': 3.3},
 {'_id': 'b97fe9ff-2312-4045-876c-b71e4f277d68', 'avgRating': 2.8},
 {'_id': '04cb0722-7fa2-43d6-bc65-0c1cd067ed3b', 'avgRating': 3.5},
 {'_id': 'ad439ff4-7fa2-43ef-aeff-fea996ca1d4f', 'avgRating': 2.2},
 {'_id': '74605146-5400-446c-8638-3e268cf0ecfc', 'avgRating': 3.9},
 {'_id': '31947696-320f-4ffb-a913-08a320e1e327', 'avgRating': 3.9},
 {'_id': 'd0023b00-35a7-4fd7-97c1-d397bda85125', 'avgRating': 3.7},
 {'_id': '0d15b819-4347-4747-897c-c259c4e2ed08', 'avgRating': 4.1},
 {'_id': '0b1598d1-e7cf-49de-9f4c-cba8d279698b', 'avgRating': 3.3}]

In [ ]:
## c. Group by course category with enrollment count
enrollment_by_category = list(db.enrollments.aggregate([
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.category",
            "totalEnrollments": {"$sum": 1}
        }
    }
]))
enrollment_by_category

[{'_id': 'Business', 'totalEnrollments': 4},
 {'_id': 'AI', 'totalEnrollments': 1},
 {'_id': 'Cybersecurity', 'totalEnrollments': 6},
 {'_id': 'Data Science', 'totalEnrollments': 4}]

In [65]:
##4.3 Student Performance Analysis
#2a. Average grade per student
avg_grade_per_student = list(db.submissions.aggregate([
    {
        "$group": {
            "_id": "$studentId",
            "averageGrade": {"$avg": "$grade"}
        }
    }
]))
avg_grade_per_student

[{'_id': 'eeae0acf-d682-433f-95dc-b74e8ca63329', 'averageGrade': 48.015},
 {'_id': '90eea846-ad23-4b3c-a66c-087d84c60470', 'averageGrade': 28.165},
 {'_id': '2bd29005-b06b-4f56-89cb-69badc24a880',
  'averageGrade': 34.306666666666665},
 {'_id': 'f08b8f8b-88cf-475c-8c7f-8006507aee04', 'averageGrade': 31.75},
 {'_id': 'e60378df-df39-4d51-932b-f7c140c62ab4', 'averageGrade': 18.81},
 {'_id': '83792cfb-a03f-4c11-ae9a-21f37fd4e85d', 'averageGrade': 50.65},
 {'_id': 'ffad8fd7-3385-401a-99d6-930d4e919c43', 'averageGrade': 87.825}]

In [66]:
##b. Completion rate by course
completion_rate_by_course = list(db.enrollments.aggregate([
    {
        "$group": {
            "_id": "$courseId",
            "totalEnrolled": {"$sum": 1},
            "completedCount": {
                "$sum": {
                    "$cond": [{"$eq": ["$status", "completed"]}, 1, 0]
                }
            }
        }
    },
    {
        "$project": {
            "completionRate": {
                "$cond": [
                    {"$eq": ["$totalEnrolled", 0]},
                    0,
                    {"$multiply": [{"$divide": ["$completedCount", "$totalEnrolled"]}, 100]}
                ]
            }
        }
    }
]))
completion_rate_by_course

[{'_id': 'b97fe9ff-2312-4045-876c-b71e4f277d68', 'completionRate': 0.0},
 {'_id': '31947696-320f-4ffb-a913-08a320e1e327',
  'completionRate': 33.33333333333333},
 {'_id': '0b1598d1-e7cf-49de-9f4c-cba8d279698b', 'completionRate': 0.0},
 {'_id': 'd0023b00-35a7-4fd7-97c1-d397bda85125', 'completionRate': 0.0},
 {'_id': '74605146-5400-446c-8638-3e268cf0ecfc', 'completionRate': 0.0}]

In [67]:
##c. Top-performing students
top_students = list(db.submissions.aggregate([
    {
        "$group": {
            "_id": "$studentId",
            "avgGrade": {"$avg": "$grade"}
        }
    },
    {"$sort": {"avgGrade": -1}},
    {"$limit": 5}
]))
top_students

[{'_id': 'ffad8fd7-3385-401a-99d6-930d4e919c43', 'avgGrade': 87.825},
 {'_id': '83792cfb-a03f-4c11-ae9a-21f37fd4e85d', 'avgGrade': 50.65},
 {'_id': 'eeae0acf-d682-433f-95dc-b74e8ca63329', 'avgGrade': 48.015},
 {'_id': '2bd29005-b06b-4f56-89cb-69badc24a880',
  'avgGrade': 34.306666666666665},
 {'_id': 'f08b8f8b-88cf-475c-8c7f-8006507aee04', 'avgGrade': 31.75}]

In [69]:
## 3. Instructor Analytics
## a. Total students taught by each instructor
students_per_instructor = list(db.enrollments.aggregate([
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.instructorId",
            "uniqueStudents": {"$addToSet": "$studentId"}
        }
    },
    {
        "$project": {
            "totalStudents": {"$size": "$uniqueStudents"}
        }
    }
]))

students_per_instructor

[{'_id': 'dd10ef06-389c-4e00-a33f-d941bc6a5cc1', 'totalStudents': 4},
 {'_id': 'df36112c-3dab-48d6-8235-5fcac6fc5fce', 'totalStudents': 4},
 {'_id': '70f51a9d-d94b-469e-a5cf-bf95f15917c4', 'totalStudents': 3},
 {'_id': '7cd1ab9a-a9a8-475b-9264-4b867caf783b', 'totalStudents': 3}]

In [71]:
## b. Average course rating per instructor
avg_rating_per_instructor = list(db.courses.aggregate([
    {
        "$unwind": "$rating"
    },
    {
        "$group": {
            "_id": "$instructorId",
            "avgRating": {"$avg": "$rating"}
        }
    }
]))
avg_rating_per_instructor

[{'_id': '70f51a9d-d94b-469e-a5cf-bf95f15917c4',
  'avgRating': 3.766666666666667},
 {'_id': 'df36112c-3dab-48d6-8235-5fcac6fc5fce', 'avgRating': 2.75},
 {'_id': '7cd1ab9a-a9a8-475b-9264-4b867caf783b',
  'avgRating': 3.5999999999999996},
 {'_id': 'dd10ef06-389c-4e00-a33f-d941bc6a5cc1',
  'avgRating': 3.3499999999999996}]

In [72]:
## c. Revenue generated per instructor
revenue_per_instructor = list(db.enrollments.aggregate([
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.instructorId",
            "totalRevenue": {"$sum": "$course.price"}
        }
    }
]))
revenue_per_instructor

[{'_id': 'dd10ef06-389c-4e00-a33f-d941bc6a5cc1',
  'totalRevenue': 434.40999999999997},
 {'_id': 'df36112c-3dab-48d6-8235-5fcac6fc5fce', 'totalRevenue': 358.32},
 {'_id': '70f51a9d-d94b-469e-a5cf-bf95f15917c4', 'totalRevenue': 117.09},
 {'_id': '7cd1ab9a-a9a8-475b-9264-4b867caf783b', 'totalRevenue': 259.86}]

In [73]:
##4. Advanced Analytics
##a. Monthly enrollment trends (last 12 months)
monthly_enrollments = list(db.enrollments.aggregate([
    {
        "$match": {
            "enrolledAt": {
                "$gte": datetime.now(timezone.utc) - timedelta(days=365)
            }
        }
    },
    {
        "$group": {
            "_id": {
                "year": {"$year": "$enrolledAt"},
                "month": {"$month": "$enrolledAt"}
            },
            "count": {"$sum": 1}
        }
    },
    {"$sort": {"_id.year": 1, "_id.month": 1}}
]))
monthly_enrollments

[{'_id': {'year': 2025, 'month': 1}, 'count': 4},
 {'_id': {'year': 2025, 'month': 2}, 'count': 1},
 {'_id': {'year': 2025, 'month': 3}, 'count': 3},
 {'_id': {'year': 2025, 'month': 4}, 'count': 4},
 {'_id': {'year': 2025, 'month': 5}, 'count': 3}]

In [74]:
##b. Most popular course categories (by enrollment count)
popular_categories = list(db.enrollments.aggregate([
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.category",
            "enrollmentCount": {"$sum": 1}
        }
    },
    {"$sort": {"enrollmentCount": -1}}
]))
popular_categories

[{'_id': 'Cybersecurity', 'enrollmentCount': 6},
 {'_id': 'Data Science', 'enrollmentCount': 4},
 {'_id': 'Business', 'enrollmentCount': 4},
 {'_id': 'AI', 'enrollmentCount': 1}]

In [75]:
##c. Student engagement metrics
avg_progress_per_student = list(db.enrollments.aggregate([
    {
        "$group": {
            "_id": "$studentId",
            "avgProgress": {"$avg": "$progress"}
        }
    }
]))
avg_progress_per_student

[{'_id': 'eeae0acf-d682-433f-95dc-b74e8ca63329', 'avgProgress': 90.33},
 {'_id': '90eea846-ad23-4b3c-a66c-087d84c60470', 'avgProgress': 52.72},
 {'_id': '2bd29005-b06b-4f56-89cb-69badc24a880',
  'avgProgress': 46.980000000000004},
 {'_id': 'f08b8f8b-88cf-475c-8c7f-8006507aee04', 'avgProgress': 41.82},
 {'_id': 'f8b895de-1167-4f7e-b57f-8fae6e18fde6',
  'avgProgress': 61.123333333333335},
 {'_id': '83792cfb-a03f-4c11-ae9a-21f37fd4e85d', 'avgProgress': 3.62},
 {'_id': 'e6d981c6-be75-4abb-9a74-c69be70fc54e', 'avgProgress': 81.32},
 {'_id': 'ffad8fd7-3385-401a-99d6-930d4e919c43', 'avgProgress': 36.15},
 {'_id': '656104b8-9f8b-4e05-bf37-db9485ed5b1b',
  'avgProgress': 20.064999999999998},
 {'_id': '2ae7b5c7-ca71-4595-b802-b2d19feaf29d', 'avgProgress': 60.43}]

### Section 5: Indexing and Performance

In [76]:
## a. Creating appropriate indexes
# Index for user email lookup (unique for quick lookup)
db.users.create_index("email", unique=True)

# Indexes for course search by title (text index) and category
db.courses.create_index([("title", "text"), ("category", 1)])

# Index for assignment queries by due date
db.assignments.create_index("dueDate")

# Compound index for enrollment queries by student and course
db.enrollments.create_index([("studentId", 1), ("courseId", 1)])


'studentId_1_courseId_1'

In [77]:
##  Task 5.2: Query Optimization
import time

start = time.time()
result = db.users.find_one({"email": "student@example.com"})
end = time.time()
print("User lookup time:", end - start)
print("Explain plan:", db.users.find({"email": "student@example.com"}).explain())


User lookup time: 0.005166769027709961
Explain plan: {'explainVersion': '1', 'queryPlanner': {'namespace': 'eduhub_db.users', 'parsedQuery': {'email': {'$eq': 'student@example.com'}}, 'indexFilterSet': False, 'queryHash': 'ED28E3D2', 'planCacheShapeHash': 'ED28E3D2', 'planCacheKey': '1E950542', 'optimizationTimeMillis': 0, 'maxIndexedOrSolutionsReached': False, 'maxIndexedAndSolutionsReached': False, 'maxScansToExplodeReached': False, 'prunedSimilarIndexes': False, 'winningPlan': {'isCached': False, 'stage': 'EXPRESS_IXSCAN', 'keyPattern': '{ email: 1 }', 'indexName': 'email_1'}, 'rejectedPlans': []}, 'executionStats': {'executionSuccess': True, 'nReturned': 0, 'executionTimeMillis': 0, 'totalKeysExamined': 0, 'totalDocsExamined': 0, 'executionStages': {'isCached': False, 'stage': 'EXPRESS_IXSCAN', 'keyPattern': '{ email: 1 }', 'indexName': 'email_1', 'keysExamined': 0, 'docsExamined': 0, 'nReturned': 0}, 'allPlansExecution': []}, 'queryShapeHash': '13B5689DF2E437921787A21BC857ED38433A

In [ ]:
# b. Search courses by title (partial match)
start = time.time()
courses = list(db.courses.find({"$text": {"$search": "python"}}))
end = time.time()
print("Course search time:", end - start)
print("Explain plan:", db.courses.find({"$text": {"$search": "python"}}).explain())


Course search time: 0.002566099166870117
Explain plan: {'explainVersion': '1', 'queryPlanner': {'namespace': 'eduhub_db.courses', 'parsedQuery': {'$text': {'$search': 'python', '$language': 'english', '$caseSensitive': False, '$diacriticSensitive': False}}, 'indexFilterSet': False, 'queryHash': 'CF6F4CEE', 'planCacheShapeHash': 'CF6F4CEE', 'planCacheKey': '08852285', 'optimizationTimeMillis': 0, 'maxIndexedOrSolutionsReached': False, 'maxIndexedAndSolutionsReached': False, 'maxScansToExplodeReached': False, 'prunedSimilarIndexes': False, 'winningPlan': {'isCached': False, 'stage': 'TEXT_MATCH', 'indexPrefix': {}, 'indexName': 'title_text_category_1', 'parsedTextQuery': {'terms': ['python'], 'negatedTerms': [], 'phrases': [], 'negatedPhrases': []}, 'textIndexVersion': 3, 'inputStage': {'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'_fts': 'text', '_ftsx': 1, 'category': 1}, 'indexName': 'title_text_category_1', 'isMultiKey': True, 'isUnique': False, 'isSparse': Fals

In [ ]:
#c. Find assignments due within a week
today = datetime.now(timezone.utc)
next_week = today + timedelta(days=7)

start = time.time()
assignments = list(db.assignments.find({
    "dueDate": {"$gte": today, "$lte": next_week}
}))
end = time.time()
print("Upcoming assignment query time:", end - start)
print("Explain plan:", db.assignments.find({
    "dueDate": {"$gte": today, "$lte": next_week}
}).explain())


Upcoming assignment query time: 0.0058040618896484375
Explain plan: {'explainVersion': '1', 'queryPlanner': {'namespace': 'eduhub_db.assignments', 'parsedQuery': {'$and': [{'dueDate': {'$lte': datetime.datetime(2025, 6, 19, 17, 16, 31, 329000)}}, {'dueDate': {'$gte': datetime.datetime(2025, 6, 12, 17, 16, 31, 329000)}}]}, 'indexFilterSet': False, 'queryHash': '63FD7581', 'planCacheShapeHash': '63FD7581', 'planCacheKey': 'F669F935', 'optimizationTimeMillis': 0, 'maxIndexedOrSolutionsReached': False, 'maxIndexedAndSolutionsReached': False, 'maxScansToExplodeReached': False, 'prunedSimilarIndexes': False, 'winningPlan': {'isCached': False, 'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'dueDate': 1}, 'indexName': 'dueDate_1', 'isMultiKey': False, 'multiKeyPaths': {'dueDate': []}, 'isUnique': False, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'dueDate': ['[new Date(1749748591329), new Date(1750353391329)]']}}}, 'reje

### Section 6: Data Validation and Error Handling

In [80]:

## 6.1 Schema Validation
# Define validation rules for each collection
validation_commands = [
    {
        "collMod": "users",
        "validator": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["userId", "email", "firstName", "lastName", "role"],
                "properties": {
                    "userId": {"bsonType": "string"},
                    "email": {
                        "bsonType": "string",
                        "pattern": "^.+@.+$"
                    },
                    "firstName": {"bsonType": "string"},
                    "lastName": {"bsonType": "string"},
                    "role": {"enum": ["student", "instructor"]},
                    "dateJoined": {"bsonType": "date"},
                    "profile": {
                        "bsonType": "object",
                        "properties": {
                            "bio": {"bsonType": "string"},
                            "avatar": {"bsonType": "string"},
                            "skills": {
                                "bsonType": "array",
                                "items": {"bsonType": "string"}
                            }
                        }
                    },
                    "isActive": {"bsonType": "bool"}
                }
            }
        },
        "validationLevel": "strict"
    },
    {
        "collMod": "courses",
        "validator": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["courseId", "title", "instructorId", "level"],
                "properties": {
                    "courseId": {"bsonType": "string"},
                    "title": {"bsonType": "string"},
                    "description": {"bsonType": "string"},
                    "instructorId": {"bsonType": "string"},
                    "category": {"bsonType": "string"},
                    "level": {"enum": ["beginner", "intermediate", "advanced"]},
                    "duration": {"bsonType": "double"},
                    "price": {"bsonType": "double"},
                    "tags": {
                        "bsonType": "array",
                        "items": {"bsonType": "string"}
                    },
                    "createdAt": {"bsonType": "date"},
                    "updatedAt": {"bsonType": "date"},
                    "isPublished": {"bsonType": "bool"},
                    "rating": {"bsonType": "double"}
                }
            }
        },
        "validationLevel": "strict"
    },
    {
        "collMod": "assignments",
        "validator": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["assignmentId", "courseId", "title", "description", "dueDate"],
                "properties": {
                    "assignmentId": {"bsonType": "string"},
                    "courseId": {"bsonType": "string"},
                    "lessonId": {"bsonType": "string"},
                    "title": {"bsonType": "string"},
                    "description": {"bsonType": "string"},
                    "dueDate": {"bsonType": "date"},
                    "points": {"bsonType": "int"},
                    "createdAt": {"bsonType": "date"}
                }
            }
        },
        "validationLevel": "strict"
    },
    {
        "collMod": "submissions",
        "validator": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["submissionId", "assignmentId", "studentId", "submittedAt"],
                "properties": {
                    "submissionId": {"bsonType": "string"},
                    "assignmentId": {"bsonType": "string"},
                    "studentId": {"bsonType": "string"},
                    "content": {"bsonType": "string"},
                    "grade": {"bsonType": "double"},
                    "feedback": {"bsonType": "string"},
                    "submittedAt": {"bsonType": "date"}
                }
            }
        },
        "validationLevel": "strict"
    },
    {
        "collMod": "enrollments",
        "validator": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["enrollmentId", "studentId", "courseId", "enrolledAt", "status"],
                "properties": {
                    "enrollmentId": {"bsonType": "string"},
                    "studentId": {"bsonType": "string"},
                    "courseId": {"bsonType": "string"},
                    "enrolledAt": {"bsonType": "date"},
                    "status": {
                        "enum": ["active", "completed", "dropped"]
                    },
                    "progress": {
                        "bsonType": "double",
                        "minimum": 0,
                        "maximum": 100
                    }
                }
            }
        },
        "validationLevel": "strict"
    },
    {
        "collMod": "lessons",
        "validator": {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["lessonId", "courseId", "title", "content", "order"],
                "properties": {
                    "lessonId": {"bsonType": "string"},
                    "courseId": {"bsonType": "string"},
                    "title": {"bsonType": "string"},
                    "content": {"bsonType": "string"},
                    "resources": {
                        "bsonType": "array",
                        "items": {"bsonType": "string"}
                    },
                    "order": {"bsonType": "int"},
                    "createdAt": {"bsonType": "date"}
                }
            }
        },
        "validationLevel": "strict"
    }
]

# Apply validation to each collection
for command in validation_commands:
    db.command(command)

print("Validation rules successfully applied to all collections.")


Validation rules successfully applied to all collections.


In [81]:
## 6.2 Error Handling
from pymongo import errors


users = db["users"]

# Ensure unique index on userId to trigger duplicate key errors
users.create_index("userId", unique=True)

# Sample user document
valid_user = {
    "userId": "user123",
    "email": "test@example.com",
    "firstName": "Jane",
    "lastName": "Doe",
    "role": "student",
    "dateJoined": datetime.now(),
    "isActive": True
}

# Duplicate userId
duplicate_user = valid_user.copy()

# Invalid data type (dateJoined should be date, not string)
invalid_type_user = valid_user.copy()
invalid_type_user["userId"] = "user124"
invalid_type_user["dateJoined"] = "2025-06-12"  # Wrong type

# Missing required field (email)
missing_field_user = valid_user.copy()
missing_field_user["userId"] = "user125"
del missing_field_user["email"]

# Insert and handle errors
for user in [valid_user, duplicate_user, invalid_type_user, missing_field_user]:
    try:
        users.insert_one(user)
        print(f"✅ Inserted user: {user['userId']}")
    except errors.DuplicateKeyError as e:
        print(f"❌ Duplicate Key Error for userId '{user['userId']}': {e.details['errmsg']}")
    except errors.WriteError as e:
        print(f"❌ Write Error for userId '{user['userId']}': {e.details['errmsg']}")
    except Exception as e:
        print(f"❌ Unexpected Error for userId '{user.get('userId', 'UNKNOWN')}': {str(e)}")


✅ Inserted user: user123
❌ Duplicate Key Error for userId 'user123': E11000 duplicate key error collection: eduhub_db.users index: email_1 dup key: { email: "test@example.com" }
❌ Write Error for userId 'user124': Document failed validation
❌ Write Error for userId 'user125': Document failed validation


In [82]:
## exporting data to a sample_data.json file"
import json


# Define all your collections
collections = ["users", "courses", "assignments", "submissions", "enrollments", "lessons"]

# Create a dictionary to hold all data
data_export = {}

for col in collections:
    data_export[col] = list(db[col].find({}, {'_id': True}))  # Include ObjectId

# Export to a single JSON file
with open("sample_data.json", "w") as f:
    json.dump(data_export, f, indent=4, default=str)

print("✅ Exported all collections to sample_data.json")


✅ Exported all collections to sample_data.json


In [84]:
## exporting schema validation

# List of collections to export validation for
collections = ["users", "courses", "assignments", "submissions", "enrollments", "lessons"]

# Dict to hold validation rules
schema_validations = {}

# Get validation rules for each collection
for collection_name in collections:
    options = db.command("listCollections", filter={"name": collection_name})
    if options["ok"] and options["cursor"]["firstBatch"]:
        coll_info = options["cursor"]["firstBatch"][0]
        validator = coll_info.get("options", {}).get("validator")
        if validator:
            schema_validations[collection_name] = validator

# Save to JSON file
with open("schema_validation.json", "w") as f:
    json.dump(schema_validations, f, indent=4)

print("✅ Schema validation exported to schema_validation.json")


✅ Schema validation exported to schema_validation.json


### Task 1: Design a Data Archiving Strategy for Old Enrollments
Soft Archiving (Mark as Archived)



In [7]:
# Define cutoff date (1 year ago)
cutoff_date = datetime.now(timezone.utc) - timedelta(days=365)

db.enrollments.update_many(
    {
        "status": {"$in": ["completed", "dropped"]},
        "enrolledAt": {"$lt": cutoff_date}
    },
    {"$set": {"archived": True}}
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

### Task 2: Implement Text Search Functionality for Course Content
Goal: Enable full-text search on fields like course title, description, and lesson content.

In [10]:
def search_courses_by_keyword(db, keyword):
    """
    Searches the 'courses' collection for documents where the keyword appears
    in either the 'title' or 'description' fields (case-insensitive).
    
    Parameters:
        db (Database): The connected MongoDB database object.
        keyword (str): The keyword to search for.
    
    Returns:
        list: List of matching course documents.
    """
    query = {
        "$or": [
            {"title": {"$regex": keyword, "$options": "i"}},
            {"description": {"$regex": keyword, "$options": "i"}}
        ]
    }

    results = list(db.courses.find(query))
    return results


In [11]:
results = search_courses_by_keyword(db, "python")

for course in results:
    print(f"{course['title']} - {course.get('description', 'No description')}")

Data Analysis with Python - Learn how to analyze data using Python.
